## Importing

In [1]:
import pandas as pd
import scipy as sp
import numpy as np

import emcee
from multiprocessing import Pool

import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
from matplotlib import font_manager
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from cycler import cycler
import corner
from typing import Any, Dict, List, Optional, Tuple, Union

import lymph

## Creating or Loading Model

In [2]:
NEW_MODEL = True

filename = "latest_extended.hdf5"

if not NEW_MODEL:
    extended_systm = lymph.utils.system_from_hdf(
    filename=filename,
    name="extended/model")
else:
    graph = {
        ('tumor', 'primary') : ['II', 'III', 'IV','VII'],
        ('lnl', 'I')         : [], 
        ('lnl', 'II')        : ['I', 'III', 'V', 'VII'], 
        ('lnl', 'III')       : ['IV'], 
        ('lnl', 'IV')        : [],
        ('lnl', 'V')         : [],
        ('lnl', 'VII')       : [],
    }
    extended_systm = lymph.Unilateral(graph=graph)

print(extended_systm)

Unilateral lymphatic system with 1 tumor(s) and 6 LNL(s).
primary-0.0%->II primary-0.0%->III primary-0.0%->IV primary-0.0%->VII II-0.0%->I II-0.0%->III II-0.0%->V II-0.0%->VII III-0.0%->IV


## Modalities

In [3]:
if NEW_MODEL:
    spsn = {"PET": [0.86, 0.79],
            "MRI": [0.63, 0.81],
            "diagnostic_consensus": [0.63, 0.81],
            "pathology": [1., 1.]}
#                         ^   ^
#                specificty   sensitivity
extended_systm.modalities = spsn

## Data

#### Method to determine the involvement of a layer in a single patient


In [4]:
import itertools

def layer_involvement_expectation(inv: Optional[int], spsns: List[tuple])-> int:
    """Determine which layers are involved given the different (conflicting) measurements:

        Args:
            inv:    For one single layer, one entry gives the measured involvement for the
                    respective modality
            spsns:  Each entry of the list is the spsn tuple of the respective modality

    """
    no_of_measurements = 0
    prob = 0
    for i , spsn in enumerate(spsns):
        if(inv[i] == True):
            no_of_measurements+=1
            prob += spsn[1]
        elif(inv[i] == False):
            no_of_measurements+=1
            prob += (1-spsn[0])
    if(no_of_measurements == 0):
        return 0
    mean_prob = prob / no_of_measurements
    if(mean_prob >= 0.5):
        return 1
    else:
        return 0


def layer_involvement_statistical(inv: Optional[int], spsns: List[tuple])-> int:
    """Determine which layers are involved given the different (conflicting) measurements:

        Args:
            inv:    For one single layer, one entry gives the measured involvement for the
                    respective modality
            spsns:  Each entry of the list is the spsn tuple of the respective modality

    """
    p_healthy = 1
    p_involved = 1
    for i , spsn in enumerate(spsns):
        if(inv[i] == True):
            p_healthy *= 1-spsn[1]
            p_involved *= spsn[1]
        elif(inv[i] == False):
            p_healthy *= spsn[0]
            p_involved *= 1-spsn[0]
    if(p_healthy < p_involved):
        return 1
    else:
        return 0


def layer_involvement_hierarchical(inv: Optional[int], spsns: List[tuple])-> int:
    """Determine which layers are involved given the different (conflicting) measurements:
       hierarchy: 1.Pathology,  2.Diagnostic consensus,  3. PET/CT,  4.MRI

        Args:
            inv:    For one single layer, one entry gives the measured involvement for the
                    respective modality. For this function the modalities have to be given in the hierachical
                    order -> pathology first etc.
            spsns:  Each entry of the list is the spsn tuple of the respective modality

    """
    spsns_with_index = []
    for i, spsn in enumerate(spsns):
        spsn_w_i = list(spsn)
        spsn_w_i.append(i)
        spsns_with_index.append(tuple(spsn_w_i))
    zipped_list = zip(spsns, inv)
    sorted_by_sp = sorted(zipped_list, key=lambda x: x[0], reverse=True)
    sorted_by_sn = sorted(zipped_list, key=lambda x: x[1], reverse=True)
    inv_by_sp = [element for _, element in sorted_by_sp]
    inv_by_sn = [element for _, element in sorted_by_sn]
    for i, invol in inv_by_sp:
        if(inv_by_sp[i] is not None):
            if(inv_by_sp[i] == inv_by_sn[i]):
                return inv_by_sp[i]
        else:
            
            pass
    return 0

#print(layer_involvement_hierarchical([0, 0, 1, 0], [(1.,1.), (0.63, 0.81), (0.86, 0.79), (0.63, 0.81)]))


# all_possible_measurements = list(itertools.product([0,1,None], repeat=4))
# for meas in all_possible_measurements:
#     exp = layer_involvement_expectation(meas, [(1.,1.), (0.63, 0.81), (0.86, 0.79), (0.63, 0.81)])
#     stat = layer_involvement_statistical(meas, [(1.,1.), (0.63, 0.81), (0.86, 0.79), (0.63, 0.81)])
#     hier = layer_involvement_hierarchical(meas, [(1.,1.), (0.63, 0.81), (0.86, 0.79), (0.63, 0.81)])
#     if(exp != stat):
#         print(meas, f"\t\t\t\texp={exp}   !=   stat={stat}")
#     if(hier != stat):
#         print(meas, f"\t\t\t\thier={hier}   !=   stat={stat}")


### Method to count the occurrences of involved layers in the real Dataset


In [5]:
if NEW_MODEL:
    data = pd.read_csv("latest.csv", header=[0,1,2] )
    t_stage = data.iloc[:,18]
    data = data.iloc[:,21:168]
    ipsi_data = data.xs("ipsi",level=1,axis=1)
    t_stage.loc[t_stage <= 2, ] = "early"
    t_stage.loc[t_stage!="early", ] = "late"
    
    
    ipsi_data["info","t_stage"]= t_stage
    ipsi_data = ipsi_data.drop(['Ia', 'Ib',"IIa","IIb"], level=1, axis=1)
    ipsi_data = ipsi_data.drop(['CT', 'FNA',"pCT"], level=0, axis=1)

    extended_systm.patient_data = ipsi_data


C:\Users\Luca\AppData\Local\Temp\ipykernel_13432\3353186479.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipsi_data["info","t_stage"]= t_stage


In [ ]:
def layer_occurence(df, t_stage=True):
    #All labels of the Columns
    layer_cols = df.columns.get_level_values(1)

    #Delete Duplicates and t-stage from list
    layers = list(dict.fromkeys(layer_cols))
    if(t_stage == True):
        layers = layers[:-1]
   
    #Create Empty DataFrame to store the occurrences
    occurr_table = pd.DataFrame(np.zeros((1, len(layers))), columns=layers)

    #Fill the occurrence table
    for layer in layers:
        select = layer_cols.isin([layer])
        level_data = df.loc[:, select]
        for index, row in level_data.iterrows():
            patient_occ = row.to_dict()
            patient_occ2 = {key[0]: val for (key, val) in patient_occ.items()}
            involved = patient_layer_involvement(patient_occ2)
            if(involved):
                occurr_table[layer] += 1
    return occurr_table

layer_occurence(ipsi_data)


#### Compare Dataset with Model

In [ ]:
def comparison_risk_w_dataset(df, layers, time_dists):

    ##layers = ["I", "IV"]
    
    # time_dists={
    #     "early": lymph.utils.fast_binomial_pmf(t, max_t, early_p),
    #     "late" : lymph.utils.fast_binomial_pmf(t, max_t, mean_late_p)
    # }
    
    all_layer_cols = df.columns.get_level_values(1)
    all_layers = list(dict.fromkeys(all_layer_cols))[:-1]
    involvement = np.repeat(None, len(all_layers))
    for i, layer in enumerate(all_layers):
        if(layers.includes(layer)):
            involvement[i]= 1
    diagnose = {"PET": [None,None,None,None,None,None]}
    
    for key in time_dists.keys():
        risk =  extended_systm.risk(
                        diagnoses=diagnose, inv=involvement,
                        time_dist=time_dists[key], 
                        mode="HMM"
                    )


## Storage of model

In [ ]:
if NEW_MODEL:
    extended_systm.to_hdf(
        filename=filename, 
        name="extended/model"
    )

## Likelihood

In [ ]:
#settings for the binom distributions
early_p=0.3
max_t=10
t = np.arange(max_t + 1)

def llh(theta):
    spread_probs, late_p = theta[:-1], theta[-1]
    
    print("run")
    if late_p > 1. or late_p < 0.:
        return -np.inf
    
    
    time_dists={
        "early": sp.stats.binom.pmf(t, max_t, early_p),
        "late" : sp.stats.binom.pmf(t, max_t, late_p),
    }
     
    return extended_systm.marginal_log_likelihood(spread_probs, t_stages=["early", "late"], time_dists=time_dists)

## Sampling

In [ ]:
# Settings for the sampler
ndim = len(extended_systm.spread_probs) + 1
nwalkers = 20 * ndim
nstep = 10000
burnin = 5000
moves = [(emcee.moves.DEMove(), 0.8), (emcee.moves.DESnookerMove(), 0.2)]


if NEW_MODEL:
    #prepare the backend
    backend = emcee.backends.HDFBackend(
        filename=filename,
        name="extended/samples"
    )
    backend.reset(nwalkers, ndim)

    
    # starting point
    initial_spread_probs = np.random.uniform(low=0., high=1., size=(nwalkers,ndim))

    
    if __name__ == "__main__":
        with Pool() as pool:
            sampler = emcee.EnsembleSampler(
                nwalkers, ndim, 
                llh, 
                moves=moves, pool=pool,
                backend=backend
            )
            sampler.run_mcmc(initial_spread_probs, nstep, progress=True)
        samples_HMM = sampler.get_chain(flat=True, discard=burnin)
        print(samples_HMM)      
else:
    recover_backend = emcee.backends.HDFBackend(filename=filename, name="extended/samples")
    samples_HMM = recover_backend.get_chain(flat=True, discard=burnin)



## Separating the Parameters


In [ ]:
spread_probs = []
late_p = []
for sample in samples_HMM:
    spread_probs.append(sample[:-1])
    late_p.append(sample[-1])

extended_systm.spread_probs = np.mean(spread_probs, axis=0)
mean_late_p = np.mean(np.array(late_p), axis=0)

## Plot Settings



In [ ]:
plt.style.use("lymph.mplstyle")

def set_size(width="single", unit="cm", ratio="golden"):
    if width == "single":
        width = 10
    elif width == "full":
        width = 32
    else:
        try:
            width = width
        except:
            width = 10

    if unit == "cm":
        width = width / 2.54

    if ratio == "golden":
        ratio = 1.618
    else:
        ratio = ratio

    try:
        height = width / ratio
    except:
        height = width / 1.618

    return (width, height)

labels = [r"$\tilde{b}_2$", r"$\tilde{b}_3$",
          r"$\tilde{b}_4$", r"$\tilde{b}_7$",
          r"$\tilde{t}_{21}$", r"$\tilde{t}_{23}$", r"$\tilde{t}_{25}$", r"$\tilde{t}_{27}$", r"$\tilde{t}_{36}$"]


fig = plt.figure(figsize=set_size(width="full", ratio=1))


# USZ colors
usz_blue = '#005ea8'
usz_green = '#00afa5'
usz_red = '#ae0060'
usz_orange = '#f17900'
usz_gray = '#c5d5db'

# colormaps
white_to_blue  = LinearSegmentedColormap.from_list("white_to_blue", 
                                                   ["#ffffff", usz_blue], 
                                                   N=256)
white_to_green = LinearSegmentedColormap.from_list("white_to_green", 
                                                   ["#ffffff", usz_green], 
                                                   N=256)
green_to_red   = LinearSegmentedColormap.from_list("green_to_red", 
                                                   [usz_green, usz_red], 
                                                   N=256)

h = usz_gray.lstrip('#')
gray_rgba = tuple(int(h[i:i+2], 16) / 255. for i in (0, 2, 4)) + (1.0,)
tmp = LinearSegmentedColormap.from_list("tmp", [usz_green, usz_red], N=128)
tmp = tmp(np.linspace(0., 1., 128))
tmp = np.vstack([np.array([gray_rgba]*128), tmp])
halfGray_halfGreenToRed = ListedColormap(tmp)

## Corner Plot


In [ ]:

corner.corner(np.array(spread_probs), labels=labels, smooth=True, fig=fig,
              hist_kwargs={'histtype': 'stepfilled', 'color': usz_blue},
              **{'plot_datapoints': False, 'no_fill_contours': True,
                 "density_cmap": white_to_blue.reversed(),
                 "contour_kwargs": {"colors": "k"},
                 "levels": np.array([0.2, 0.5, 0.8])},
              show_titles=True, title_kwargs={"fontsize": "medium"})

axes = fig.get_axes()
for ax in axes:
    ax.grid(False)


plt.savefig("corner_HMM.png", dpi=300, bbox_inches="tight")
plt.savefig("corner_HMM.svg", bbox_inches="tight")

## Transition Matrix


In [ ]:

# modify the transition matrix for nicer coloring
mod_A = -1 * np.ones_like(extended_systm.A)
for key, nums in extended_systm.mask.items():
    for i in nums:
        mod_A[key, i] = extended_systm.A[key, i]

# Divide the Matrix into three submatrices
x_values = [extended_systm.state_list[:32],extended_systm.state_list[32:], extended_systm.state_list[32:]]
y_values = [extended_systm.state_list[:32],extended_systm.state_list[:32], extended_systm.state_list[32:]]
partA = [mod_A[:32,:32], mod_A[:32,32:], mod_A[32:,32:]]
name = ["1", "2","3"]

#Create the plot for each submatrix
for k , val in enumerate(x_values):
    fig, ax = plt.subplots(figsize=set_size(ratio=1., width="full"),
                       constrained_layout=True)

    h = ax.imshow(partA[k], cmap=halfGray_halfGreenToRed, vmin=-1., vmax=1.)
    ax.set_xticks(range(len(x_values[k])))
    ax.set_xticklabels(x_values[k], rotation=-90, fontsize=20)
    ax.set_yticks(range(len(y_values[k])))
    ax.set_yticklabels(y_values[k], fontsize=20)
    ax.tick_params(direction="out")
    ax.grid(False)

    # label the non-zero entries with their probability in %
    for i in range(len(x_values[k])):
        for j in range(len(y_values[k])):
            if mod_A[i, j] >= 0.:
                ax.text(j, i, f"{partA[k][i,j]*100:.1f}", ha="center", va="center",
                        color="white", fontsize=14)

    plt.savefig(f"transition_matrix{name[k]}.png", dpi=300, bbox_inches="tight")
    plt.savefig(f"Transition_matrix{name[k]}.svg", bbox_inches="tight")
    plt.clf()

## Risk prediction

In [ ]:
time_dists={
        "early": lymph.utils.fast_binomial_pmf(t, max_t, early_p),
        "late" : lymph.utils.fast_binomial_pmf(t, max_t, mean_late_p)
    }

layers = ["V", "VII"]

diagnoses_text = [["Layer V is negative", "Layer V is negative but II positive"], ["Layer VII is negative", "Layer VII is negative but II & III positive"]]

diagnoses = [[{"PET":  np.array([None,None,None,None,0,None])},
            {"PET": np.array([None,1,None,None,0,None])}],
            [{"PET": np.array([None,None,None,None,None,0])},
            {"PET": np.array([None,1,1,None,None,0])}]
            ]

involvements = [np.array([None,None,None,None,1,None]), np.array([None,None,None,None,None,1])]
thin = 50

print("Probability p for Binomial distribution of late T_stage:", round(mean_late_p,4))


for key in time_dists.keys():
    print("T_stage = ", key)
    for i, layer in enumerate(layers):
        for k, diagnose in enumerate(diagnoses[i]):
            risk =  extended_systm.risk(
                diagnoses=diagnose, inv=involvements[i],
                time_dist=time_dists[key], 
                mode="HMM"
            )
            print(f"Risk for Layer {layer} given {diagnoses_text[i][k]}:", round(risk,4))

   
